In [8]:
import pyodbc 
print(pyodbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'SQL Server Native Client 11.0', 'ODBC Driver 17 for SQL Server']


In [9]:
import pypyodbc as odbc
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc

In [10]:
# Cấu hình kết nối
DRIVER = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = 'MSI'
DATABASE_NAME = 'DATA_For_ML'
USER_NAME = 'sa'
PASSWORD = '12345'


# Chuỗi kết nối
connection_string = f"""
DRIVER={{{DRIVER}}};
SERVER={SERVER_NAME};
DATABASE={DATABASE_NAME};
UID={USER_NAME};
PWD={PASSWORD};
"""

In [11]:
# Hàm kết nối và tải dữ liệu
def load_data():
    try:
        connection = odbc.connect(connection_string)
        print("Kết nối thành công!")
       
        df_brand = pd.read_sql("SELECT * FROM Tiki_Brand", connection)
        df_comment = pd.read_sql("SELECT * FROM Tiki_Comment", connection)
        df_customer = pd.read_sql("SELECT * FROM Tiki_Customer", connection)
        df_product = pd.read_sql("SELECT * FROM Tiki_Product", connection)
        connection.close()
       
        print("Đã tải dữ liệu và ngắt kết nối.")
        return df_brand, df_comment, df_customer, df_product
    except Exception as e:
        print("Kết nối thất bại:", e)
        return None, None, None, None

In [12]:
# Hàm xử lý và phân tích dữ liệu
def analyze_data(df_brand, df_comment, df_customer, df_product):
    # Chuyển kiểu dữ liệu và loại bỏ phần .0 ở cột brand_id
    df_product['brand_id'] = df_product['brand_id'].astype(str).str.replace(r'\.0$', '', regex=True)
    df_brand['brand_id'] = df_brand['brand_id'].astype(str)


    # Tính toán các giá trị trung bình và tổng doanh thu
    avg_price = df_product['price'].mean()
    avg_rating = df_product['rating_avg'].mean()
   
    df_sales = df_comment.merge(df_product, on='product_id')
    df_sales['Revenue'] = df_sales['price']
    total_revenue = df_sales['Revenue'].sum()
   
    print(f"Giá trung bình: {avg_price}")
    print(f"Rating trung bình: {avg_rating}")
    print(f"Tổng doanh thu: {total_revenue}")


    # Merge dữ liệu để hiển thị biểu đồ
    df_rating = df_product.merge(df_brand, on='brand_id', how='inner')
    df_comment_with_price = df_comment.merge(df_product[['product_id', 'price']], on='product_id', how='left')
    df_comment_with_price['revenue_per_product'] = df_comment_with_price.groupby('product_id')['product_id'].transform('size') * df_comment_with_price['price']
    df_sales_with_name = df_comment_with_price.merge(df_customer, on='customer_id', how='left')
    df_customer_revenue = df_sales_with_name.groupby(['customer_id', 'customer_name']).agg({'revenue_per_product': 'sum'}).reset_index()
   
    return df_rating, df_customer_revenue, df_sales, df_comment, avg_price, avg_rating, total_revenue

In [13]:
# Hàm tạo biểu đồ bằng Plotly
def create_plots(df_rating, df_customer_revenue, df_sales, df_comment, df_product):
    # Biểu đồ số lượng rating theo từng danh mục
    ratings_by_category = (
        df_rating.groupby('category')
        .size()
        .reset_index(name='Rating_Count')
        .sort_values(by='Rating_Count', ascending=False)
    )
    fig1 = px.bar(ratings_by_category, x='category', y='Rating_Count', title="Số lượng rating từng danh mục",
                  color='Rating_Count', color_continuous_scale='YlOrBr')


    # Biểu đồ số lượng rating theo từng hãng sản xuất
    ratings_by_brand = (
        df_rating.groupby('brand_name')
        .size()
        .reset_index(name='Rating_Count')
        .sort_values(by='Rating_Count', ascending=False)
        .head(10)
    )
    fig2 = px.bar(ratings_by_brand, x='brand_name', y='Rating_Count', title="Số lượng rating từng hãng sản xuất<br>(Top 10)",
                  color='Rating_Count', color_continuous_scale='YlOrBr')


    # Biểu đồ top khách hàng mang lại nhiều doanh thu
    top_customers = df_customer_revenue.nlargest(10, 'revenue_per_product').sort_values(by='revenue_per_product', ascending=False)
    fig3 = px.bar(top_customers, x='customer_name', y='revenue_per_product', title="Top 10 khách hàng<br>mang lại nhiều doanh thu",
                  color='revenue_per_product', color_continuous_scale='YlOrBr')


    # Biểu đồ số lượng khách hàng theo thời gian
    df_comment['purchased_at'] = pd.to_datetime(df_comment['purchased_at'], errors='coerce')
    df_comment = df_comment.dropna(subset=['purchased_at'])
    df_comment_2019 = df_comment[df_comment['purchased_at'].dt.year == 2019]
   
    customers_by_time_2019 = (
        df_comment_2019.groupby(df_comment_2019['purchased_at'].dt.to_period('M'))
        .size()
        .reset_index(name='Customer_Count')
    )
    customers_by_time_2019['purchased_at'] = customers_by_time_2019['purchased_at'].dt.to_timestamp()
   
    fig4 = px.line(customers_by_time_2019, x='purchased_at', y='Customer_Count', title="Lượng khách hàng theo thời gian<br>(Năm 2019)",
                   color_discrete_sequence=['orange'])


    # Biểu đồ danh mục sản phẩm được mua nhiều theo thời gian
    df_sales_filtered = df_sales[df_sales['category'] == 'Nhà Cửa - Đời Sống']
    df_sales_filtered['purchased_at'] = pd.to_datetime(df_sales_filtered['purchased_at'], errors='coerce')
    df_sales_filtered = df_sales_filtered.dropna(subset=['purchased_at'])
    df_sales_filtered = df_sales_filtered[df_sales_filtered['purchased_at'].dt.year == 2019]
   
    products_by_category_time = df_sales_filtered.groupby([df_sales_filtered['purchased_at'].dt.to_period('M')]).size().reset_index(name='Product_Count')
    products_by_category_time['purchased_at'] = products_by_category_time['purchased_at'].dt.to_timestamp()
   
    fig5 = px.line(products_by_category_time, x='purchased_at', y='Product_Count', title="Sản phẩm 'Nhà Cửa - Đời Sống'<br>được mua nhiều theo thời gian (2019)",
                   color_discrete_sequence=['orange'])


    # Biểu đồ xu hướng đánh giá trung bình theo tháng
    trend_df = df_comment_2019.groupby(df_comment_2019['purchased_at'].dt.to_period('M'))['rating'].mean().reset_index()
    trend_df['Date'] = trend_df['purchased_at'].dt.to_timestamp()
    trend_df.columns = ['Month', 'Average Rating', 'Date']
   
    fig6 = px.line(trend_df, x='Date', y='Average Rating', title='Xu hướng đánh giá trung bình<br>theo từng tháng (2019)',
                   color_discrete_sequence=['orange'])


    # Biểu đồ số lượng sản phẩm theo từng danh mục
    products_by_category = (
        df_product.groupby('category')
        .size()
        .reset_index(name='Product_Count')
        .sort_values(by='Product_Count', ascending=False)
    )
   
    fig7 = px.bar(products_by_category, x='category', y='Product_Count', title='Số lượng sản phẩm từng danh mục',
                  color='Product_Count', color_continuous_scale='YlOrBr')


    # Cập nhật nền và tiêu đề của các biểu đồ
    figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7]
    for fig in figures:
        fig.update_layout(
            plot_bgcolor='black',
            paper_bgcolor='black',
            font=dict(color='white', size=15, family='Arial'),
            title=dict(
                text=fig.layout.title.text,
                font=dict(color='white', size=20, family='Arial'),
                x=0.5,
                xanchor='center',
            )
        )
   
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7

In [14]:
# Chạy các hàm và tạo biểu đồ
df_brand, df_comment, df_customer, df_product = load_data()


if df_brand is not None:
    df_rating, df_customer_revenue, df_sales, df_comment, avg_price, avg_rating, total_revenue = analyze_data(df_brand, df_comment, df_customer, df_product)
    fig1, fig2, fig3, fig4, fig5, fig6, fig7 = create_plots(df_rating, df_customer_revenue, df_sales, df_comment, df_product)

Kết nối thành công!


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_28736\2685557278.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_brand = pd.read_sql("SELECT * FROM Tiki_Brand", connection)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_28736\2685557278.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_comment = pd.read_sql("SELECT * FROM Tiki_Comment", connection)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_28736\2685557278.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customer = pd.read_sql("SELECT * FROM Tiki_Customer", con

Đã tải dữ liệu và ngắt kết nối.
Giá trung bình: 722206.741786847
Rating trung bình: 3.461737678207248
Tổng doanh thu: 110043324079.0


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_28736\1582971062.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
# Tạo và hiển thị dashboard
app = Dash(__name__)


app.layout = html.Div([
    html.H1("Dashboard Phân Tích Dữ Liệu Tiki", style={'color': 'white', 'textAlign': 'center', 'font-size': '30px'}),
    html.Div([
        html.P(f"Giá trung bình: {avg_price:.0f} VND  |  Rating trung bình: {avg_rating:.2f}  |  Tổng doanh thu: {total_revenue:.0f} VND", style={'color': 'white'}),
    ], style={'textAlign': 'center', 'color': 'white', 'font-size': '20px'}),
    html.Div([
        dcc.Graph(figure=fig1, style={'width': '50%', 'display': 'inline-block', 'height': '500px'}),
        dcc.Graph(figure=fig3, style={'width': '50%', 'display': 'inline-block', 'height': '500px'}),
    ]),
    html.Div([
        dcc.Graph(figure=fig4, style={'width': '33%', 'display': 'inline-block', 'height': '500px'}),
        dcc.Graph(figure=fig5, style={'width': '34%', 'display': 'inline-block', 'height': '500px'}),
        dcc.Graph(figure=fig6, style={'width': '33%', 'display': 'inline-block', 'height': '500px'}),
    ]),
    html.Div([
        dcc.Graph(figure=fig7, style={'width': '50%', 'display': 'inline-block', 'height': '500px'}),
        dcc.Graph(figure=fig2, style={'width': '50%', 'display': 'inline-block', 'height': '500px'}),
    ]),
], style={'backgroundColor': 'black'})


if __name__ == '__main__':
    app.run_server(debug=True)